In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

In [43]:
df_2000 = df[df['yearOfLoss']>2000]

In [44]:
#Zipcode list

list_zipcode = df_2000['reportedZipCode'].dropna().drop_duplicates().astype(int).astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_zipcode = [zipcode.zfill(5) for zipcode in list_zipcode]

In [5]:
#CensusBG list

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

## BlockGroup shapefile analysis

In [6]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 400000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
merged_gdf = pd.concat(gdf_list, ignore_index=True)

In [7]:
merged_gdf.head()

,GEOID,geometry,year
0,020900009002,"POLYGON ((-147.96856 64.77841, -147.96852 64.7...",2021
1,021300002002,"POLYGON ((-131.70187 55.38841, -131.70029 55.3...",2021
2,020200021003,"POLYGON ((-149.92843 61.18308, -149.92817 61.1...",2021
3,020200017022,"POLYGON ((-149.76341 61.19541, -149.76341 61.1...",2021
4,021300003003,"POLYGON ((-131.64374 55.33651, -131.64326 55.3...",2021


In [8]:
df_test = merged_gdf['GEOID'].value_counts().reset_index()
df_test.columns = ['GEOID', 'count']

In [9]:
df_test[df_test['count'] >= 6].count()

GEOID    37
count    37
dtype: int64

In [10]:
df_test

,GEOID,count
0,780309604001,7
1,130390102003,7
2,530299718001,7
3,780309605004,7
4,530579501001,7
...,...,...
106776,060470009032,1
106777,060470016031,1
106778,060650418033,1
106779,060371438011,1


In [105]:
list_censusBG_3 = merged_gdf['GEOID'].dropna().drop_duplicates().tolist()

In [104]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 11-digit BG ids into 12-digits
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [106]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_3)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

Number of common BG: 106781


In [107]:
len(common_bg)/len(list_censusBG)

0.999953177382803

In [112]:
# Assuming you have defined list_censusBG and common_bg
list_BG_not_found = set(list_censusBG) - common_bg
list_BG_not_found = list(list_BG_not_found)

print("CensusBG values not in common:", list_BG_not_found)

CensusBG values not in common: ['420034961020', '360594169000', '390230000000', '120853819005', '482013428000']


In [122]:
df_BG_not_found = df[['censusBlockGroupFips', 'id']]

In [125]:
df_cleaned = df_BG_not_found.dropna(subset=['censusBlockGroupFips'])

# Now you can proceed with the remaining operations on df_cleaned
df_cleaned['censusBlockGroupFips'] = [str(int(float(i))) for i in df_cleaned['censusBlockGroupFips']]
df_cleaned['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_cleaned['censusBlockGroupFips']]

C:\Users\Asus\AppData\Local\Temp\ipykernel_17020\905567320.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['censusBlockGroupFips'] = [str(int(float(i))) for i in df_cleaned['censusBlockGroupFips']]
C:\Users\Asus\AppData\Local\Temp\ipykernel_17020\905567320.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['censusBlockGroupFips'] = [censusBG.zfill(12) for censusBG in df_cleaned['censusBlockGroupFips']]


In [126]:
df_cleaned[df_cleaned['censusBlockGroupFips'].isin(list_BG_not_found)].shape[0]

8

## Zipcode shapefile analysis (until 2000)

In [11]:
chunk_size = 25000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 100000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/zipcode_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)
    
# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
merged_gdf_zipcode = pd.concat(gdf_list, ignore_index=True)

In [12]:
merged_gdf_zipcode.shape[0]

90165

In [13]:
merged_gdf_zipcode.head()

,ZIPcode,geometry,year
0,38739,"MULTIPOLYGON (((-90.49925 34.05322, -90.49972 ...",2000
1,38767,"POLYGON ((-90.76477 34.11951, -90.76698 34.119...",2000
2,38669,"POLYGON ((-90.74081 34.18456, -90.74115 34.184...",2000
3,38630,"MULTIPOLYGON (((-90.67205 34.25956, -90.67216 ...",2000
4,38639,"POLYGON ((-90.45646 34.31975, -90.45676 34.319...",2000


In [45]:
list_zip = merged_gdf_zipcode['ZIPcode'].dropna().drop_duplicates().astype(int).astype(str).tolist()

list_zip = [zipcode.zfill(5) for zipcode in list_zip]

In [46]:
list_zip_1 = list(set(list_zip))

In [47]:
set_zipcode = set(list_zipcode)
set_zipcode_3 = set(list_zip_1)

# Find the intersection of the two sets
common_zipcodes = set_zipcode.intersection(set_zipcode_3)

# Print the number of common zip codes
print("Number of common zip codes:", len(common_zipcodes))

Number of common zip codes: 20236


In [48]:
len(set_zipcode_3)

22849

In [49]:
len(set_zipcode)

21161

In [95]:
1-len(common_zipcodes)/len(set_zipcode)

0.043712489957941525

In [52]:
# Find the difference between set_zipcode and common_zipcodes
zipcodes_not_in_common = set_zipcode - common_zipcodes

# Convert the result back to a list
list_zipcodes_not_in_common = list(zipcodes_not_in_common)

In [96]:
# Convert the list of zipcodes to a set for faster membership checking
set_zipcodes_not_in_common = set(list_zipcodes_not_in_common)

df_test = df_2000[['reportedZipCode', 'yearOfLoss']]

In [97]:
df_test['reportedZipCode'] = df_test['reportedZipCode'].dropna().astype(int).astype(str)

C:\Users\Asus\AppData\Local\Temp\ipykernel_17020\957450463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['reportedZipCode'] = df_test['reportedZipCode'].dropna().astype(int).astype(str)


In [100]:
year_list = df_test[df_test['reportedZipCode'].isin(zipcodes_not_in_common)]['yearOfLoss'].unique()

In [102]:
year_list

array([2005, 2012, 2004, 2003, 2006, 2011, 2013, 2009, 2017, 2008, 2001,
       2015, 2016, 2018, 2002, 2007, 2010, 2014, 2019, 2020, 2021, 2022,
       2023], dtype=int64)